# Интеллектуальный анализ данных – весна 2024
# Домашнее задание 6: классификация текстов

Правила:



*   Домашнее задание оценивается в 10 баллов.
*   Можно использовать без доказательства любые результаты, встречавшиеся на лекциях или семинарах по курсу, если получение этих результатов не является вопросом задания.
*  Можно использовать любые свободные источники с *обязательным* указанием ссылки на них.
*  Плагиат не допускается. При обнаружении случаев списывания, 0 за работу выставляется всем участникам нарушения, даже если можно установить, кто у кого списал.
*  Старайтесь сделать код как можно более оптимальным. В частности, будет штрафоваться использование циклов в тех случаях, когда операцию можно совершить при помощи инструментов библиотек, о которых рассказывалось в курсе.

В этом домашнем задании вам предстоит построить классификатор текстов.

Будем предсказывать эмоциональную окраску твиттов о коронавирусе.



In [102]:
import numpy as np
import pandas as pd
from typing import List
import matplotlib.pyplot as plt
import seaborn as sns
from string import punctuation
import warnings

warnings.filterwarnings("ignore")

In [103]:
df = pd.read_csv(
    'https://raw.githubusercontent.com/hse-ds/iad-intro-ds/master/2024/homeworks/hw06_texts/tweets_coronavirus.csv',
    encoding='latin-1')
df.sample(4)

,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment
29060,39465,84417,Chichester and London,09-04-2020,Asian markets improve on hopes that the peak o...,Extremely Positive
30341,41082,86034,NaN,10-04-2020,"When COVID-19 Deferrals End, Better Collection...",Positive
26472,36233,81185,"Reading, UK",07-04-2020,How much will #coronavirus impact consumer beh...,Extremely Positive
15740,22935,67887,"Edmonton, Alberta",24-03-2020,Unsung heroes Grocery store workers compensate...,Positive


Для каждого твитта указано:


*   UserName - имя пользователя, заменено на целое число для анонимности
*   ScreenName - отображающееся имя пользователя, заменено на целое число для анонимности
*   Location - местоположение
*   TweetAt - дата создания твитта
*   OriginalTweet - текст твитта
*   Sentiment - эмоциональная окраска твитта (целевая переменная)



## Задание 1 Подготовка (0.5 балла)

Целевая переменная находится в колонке `Sentiment`.  Преобразуйте ее таким образом, чтобы она стала бинарной: 1 - если у твитта положительная эмоциональная окраска и 0 - если отрицательная.

In [104]:
df["Sentiment"] = df["Sentiment"].replace(
    {"Positive": 1, "Extremely Positive": 1, "Negative": 0, "Extremely Negative": 0})
df.sample(4)

,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment
14563,21497,66449,Manchester,23-03-2020,An excellent description from of the six thres...,1
26500,36270,81222,Kenya,07-04-2020,It must have been a difficult decision to make...,0
8061,13593,58545,"Marble Falls, Texas",20-03-2020,Highland Lakes area food pantries struggle to ...,0
28012,38152,83104,"Dubai, United Arab Emirates",08-04-2020,"SOMA Hand Sanitizer Add-On-Kit. Attach to ATM,...",1


Сбалансированы ли классы?

In [105]:
print(f"Положительная окраска {(df["Sentiment"] == 1).sum(axis=0)}")
print(f"Отрицательная окраска {(df["Sentiment"] == 0).sum(axis=0)}")
# Не идеально, но сбалансированные. 17 процентов разницы между классами должно быть нормально

Положительная окраска 18046
Отрицательная окраска 15398


**Ответ:** # -- YOUR ANSWER HERE --

Выведете на экран информацию о пропусках в данных. Если пропуски присутствуют заполните их строкой 'Unknown'.

In [106]:
display(df.isnull().sum())
# Есть пропуски в столбце местоположения (Location)
df = df.fillna("Unknown")
display(df.isnull().sum())

UserName            0
ScreenName          0
Location         7049
TweetAt             0
OriginalTweet       0
Sentiment           0
dtype: int64

UserName         0
ScreenName       0
Location         0
TweetAt          0
OriginalTweet    0
Sentiment        0
dtype: int64

Разделите данные на обучающие и тестовые в соотношении 7 : 3 и `random_state=0`

In [107]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df.drop(["Sentiment"], axis=1), df["Sentiment"], test_size=0.3,
                                                    random_state=0)
#stratify=df["Sentiment"] убрал стратифай, так как про него не сказано. Но стоило бы его добавить
display(X_train.sample(4))
display(X_test.sample(4))
display(y_train.sample(4))
print(f"Положительная окраска {(y_train == 1).sum(axis=0)}")  # убедиться, что стратифай оставляет баланс классов
print(f"Отрицательная окраска {(y_train == 0).sum(axis=0)}")
display(y_test.sample(4))
print(f"Положительная окраска {(y_test == 1).sum(axis=0)}")
print(f"Отрицательная окраска {(y_test == 0).sum(axis=0)}")

,UserName,ScreenName,Location,TweetAt,OriginalTweet
17297,24846,69798,Boston,25-03-2020,Thank you to our first responders healthcare w...
2521,6854,51806,"Pakistan, Karachi",18-03-2020,#COVID2019 Will kill more people economically ...
33097,44510,89462,vagabond ??(???????) 3?,13-04-2020,"#????\r\r\nFor grocery store workers, ÃÂa pa..."
24139,33306,78258,Unknown,05-04-2020,#CoronavirusPandemic #coronavirus \r\r\nWhen a...


,UserName,ScreenName,Location,TweetAt,OriginalTweet
13170,19777,64729,United States,22-03-2020,@NYGovCuomo @NYGovCuomo Need your support to p...
28167,38342,83294,United States,08-04-2020,Uneven February Revenues Must Carry Sportsbook...
20615,28957,73909,"Sheffield, England",30-03-2020,Petition started to ask supermarkets to set up...
2832,7222,52174,North America,18-03-2020,interview w shows you amp Kildee understand im...


27840    0
33349    0
4735     0
3326     1
Name: Sentiment, dtype: int64

Положительная окраска 12564
Отрицательная окраска 10846


24052    0
12008    0
1341     1
17901    1
Name: Sentiment, dtype: int64

Положительная окраска 5482
Отрицательная окраска 4552


## Задание 2 Токенизация (3 балла)

Постройте словарь на основе обучающей выборки и посчитайте количество встреч каждого токена с использованием самой простой токенизации - деления текстов по пробельным символам и приведение токенов в нижний регистр

In [108]:
from sklearn.feature_extraction.text import CountVectorizer
from collections import Counter

# Токены я брал только из столбца самих текстов OriginalTweet, так как остальные столбцы не несут в себе эмоциональной окраски

dictionary = Counter(item.lower() for ls in map(lambda x: x.split(), X_train.OriginalTweet) for item in ls)
display(dictionary)

Counter({'the': 26815,
         'to': 23373,
         'and': 14684,
         'of': 13012,
         'a': 11737,
         'in': 11198,
         'for': 8566,
         '#coronavirus': 8223,
         'is': 7383,
         'are': 7050,
         'you': 5467,
         'on': 5452,
         'i': 5340,
         'at': 4642,
         'this': 4581,
         'with': 4063,
         'prices': 3891,
         'food': 3820,
         'we': 3787,
         'have': 3770,
         'that': 3741,
         'as': 3694,
         'be': 3570,
         'grocery': 3469,
         'supermarket': 3288,
         'people': 3175,
         'covid-19': 3173,
         'store': 3155,
         'it': 3150,
         'from': 3045,
         'all': 2808,
         'your': 2784,
         'will': 2726,
         'not': 2714,
         '#covid19': 2471,
         'our': 2460,
         'my': 2445,
         '&amp;': 2314,
         'they': 2309,
         'has': 2304,
         'consumer': 2245,
         'by': 2236,
         'or': 2234,
         '

Какой размер словаря получился?

In [109]:
len(dictionary)

79755

Выведите 10 самых популярных токенов с количеством встреч каждого из них. Объясните, почему именно эти токены в топе

In [110]:
dictionary.most_common(10)
# самыми популярными оказались токены артиклей и предлогов. То есть стоп-слова, не несущие информации

[('the', 26815),
 ('to', 23373),
 ('and', 14684),
 ('of', 13012),
 ('a', 11737),
 ('in', 11198),
 ('for', 8566),
 ('#coronavirus', 8223),
 ('is', 7383),
 ('are', 7050)]

**Ответ:** самыми популярными оказались токены артиклей и предлогов. То есть стоп-слова, не несущие информации

Удалите стоп-слова из словаря и выведите новый топ-10 токенов (и количество встреч) по популярности.  Что можно сказать о нем?

In [111]:
import nltk
from nltk.corpus import stopwords

nltk.download("stopwords", quiet=True)

True

In [112]:
words = [item.lower() for ls in map(lambda x: x.split(), X_train.OriginalTweet) for item in ls]
# сделаю из списка слов множество для адекватной асимптотики
stopwords_set = set(stopwords.words("english"))
dictionary = Counter(word for word in words if word not in stopwords_set)
display(dictionary)

Counter({'#coronavirus': 8223,
         'prices': 3891,
         'food': 3820,
         'grocery': 3469,
         'supermarket': 3288,
         'people': 3175,
         'covid-19': 3173,
         'store': 3155,
         '#covid19': 2471,
         '&amp;': 2314,
         'consumer': 2245,
         '19': 2111,
         'covid': 1769,
         'get': 1759,
         '#covid_19': 1660,
         'online': 1648,
         'shopping': 1621,
         'like': 1550,
         'need': 1549,
         'panic': 1495,
         '-': 1429,
         'hand': 1382,
         'demand': 1345,
         '?': 1320,
         'help': 1282,
         'us': 1247,
         'go': 1245,
         'workers': 1242,
         'going': 1141,
         'time': 1136,
         'one': 1074,
         'sanitizer': 1064,
         'stock': 1063,
         'please': 1061,
         'due': 1059,
         'pandemic': 1026,
         'coronavirus': 970,
         'home': 959,
         '#covid2019': 915,
         'buying': 911,
         'oil': 9

In [113]:
len(dictionary)  # удалили 171 уникальный токен

79584

In [114]:
dictionary.most_common(10)
# Новые слова уже более осмысленные. Конечно также встречается шум для задачи определения настроения, связанный с коронавирусом, но также есть и важные слова вроде цен, магазинов и продуктов, дающие информацию о возможном повышении цен или карантине

[('#coronavirus', 8223),
 ('prices', 3891),
 ('food', 3820),
 ('grocery', 3469),
 ('supermarket', 3288),
 ('people', 3175),
 ('covid-19', 3173),
 ('store', 3155),
 ('#covid19', 2471),
 ('&amp;', 2314)]

**Ответ:** Новые слова уже более осмысленные конечно также встречается шум для задачи определения настроения, связанный с коронавирусом, но также есть и важные слова вроде цен, магазинов и продуктов, дающие информацию о возможном повышении цен или карантине

Также выведите 20 самых непопулярных слов (если самых непопулярных слов больше выведите любые 20 из них) Почему эти токены непопулярны, требуется ли как-то дополнительно работать с ними?

In [115]:
dictionary.most_common()[:-20:-1]
# Самых популярных слов гораздо больше 20. Часто они содержат ссылки, хештеги, теги, синтаксические ошибки или просто слова с уникальным символом после него. Думаю стоит отделить символы от слов, чтобы слова с символами не были уникальными токенами. Также регулярными выражениями можно легко удалить ссылки, теги и хештеги.

[('https://t.co/7j2y3rsld9', 1),
 ('@680news', 1),
 ('@torontopearson', 1),
 ('nothing...', 1),
 ('flew', 1),
 ('wage!', 1),
 ('renewing', 1),
 ('playersã\x82â\x92', 1),
 ('subsidise', 1),
 ('non-playing', 1),
 ('premiership', 1),
 ('https://t.co/iaek4fwsgz', 1),
 ('@srinivasiyc', 1),
 ('https://t.co/zjcl195vqs', 1),
 ('#southkerry', 1),
 ('#skelligcoast2kms', 1),
 ('@skelligsix18', 1),
 ('closer!)', 1),
 ('coast!', 1)]

**Ответ:** Самых популярных слов гораздо больше 20. Часто они содержат ссылки, хештеги, теги, синтаксические ошибки или просто слова с уникальным символом после него. Думаю стоит отделить символы от слов, чтобы слова с символами не были уникальными токенами. Также регулярными выражениями можно легко удалить ссылки, теги и хештеги.

Теперь воспользуемся токенайзером получше - TweetTokenizer из библиотеки nltk. Примените его и посмотрите на топ-10 популярных слов. Чем он отличается от топа, который получался раньше? Почему?

In [116]:
from nltk.tokenize import TweetTokenizer

tw = TweetTokenizer()
dictionary = Counter([item.lower() for ls in map(lambda x: tw.tokenize(x), X_train.OriginalTweet) for item in ls])
dictionary.most_common(10)
# отличается тем, что теперь в топе есть не только стоп-слов (оставил, так как их нужно убрать в следующем задании), но и пунктуацию, которая раньше была частью других слов.

[('the', 26993),
 ('.', 24118),
 ('to', 23478),
 (',', 17571),
 ('and', 14825),
 ('of', 13044),
 ('a', 11891),
 ('in', 11348),
 ('?', 9524),
 ('#coronavirus', 8808)]

**Ответ:** отличается тем, что теперь в топе есть не только стоп-слов (оставил, так как их нужно убрать в следующем задании), но и знаки препинания, которые раньше были частью других слов.-

Удалите из словаря стоп-слова и пунктуацию, посмотрите на новый топ-10 слов с количеством встреч, есть ли теперь в нем что-то не похожее на слова?

In [117]:
from string import punctuation

words = [item.lower() for ls in map(lambda x: tw.tokenize(x), X_train.OriginalTweet) for item in ls]
# сделаю из списка слов множество для адекватной асимптотики
stopwords_set = set(stopwords.words("english") + list(punctuation))
dictionary = Counter(word for word in words if word not in stopwords_set)

In [118]:
dictionary.most_common(10)
# Да, теперь есть странная буковка â, и символы, начинающиеся с \ (например \x82). Скорее всего это не прогрузившийся символ юникода или смайлик. Также встречаются число (тоже шум, так как это серийный номер ковида)

[('#coronavirus', 8808),
 ('â', 7415),
 ('\x82', 7311),
 ('19', 7167),
 ('covid', 6253),
 ('prices', 4601),
 ('\x92', 4372),
 ('food', 4367),
 ('store', 3877),
 ('supermarket', 3805)]

**Ответ:** Да, теперь есть странная буковка â, и символы, начинающиеся с \ (например \x82). Скорее всего это смайлик. Также встречаются число (тоже шум, так как это серийный номер ковида)

Удалите из словаря токены из одного символа, с позицией в таблице Unicode 128 и более (`ord(x) >= 128`)

Выведите топ-10 самых популярных и топ-20 непопулярных слов. Чем полученные топы отличаются от итоговых топов, полученных при использовании токенизации по пробелам? Что теперь лучше, а что хуже?

In [120]:
words = [item.lower() for ls in map(lambda x: tw.tokenize(x), X_train.OriginalTweet) for item in ls]
# сделаю из списка слов множество для адекватной асимптотики
stopwords_set = set(stopwords.words("english") + list(punctuation))
# довольно странно сформулированное задание. Удалю символы с ord(symbol) >= 128
dictionary = Counter(word for word in words if
                     ((word not in stopwords_set) and ((len(word) > 1) or (len(word) == 1 and (ord(word) < 128)))))

In [121]:
dictionary.most_common(10)

[('#coronavirus', 8808),
 ('19', 7167),
 ('covid', 6253),
 ('prices', 4601),
 ('food', 4367),
 ('store', 3877),
 ('supermarket', 3805),
 ('grocery', 3523),
 ('people', 3463),
 ('#covid19', 2589)]

In [122]:
dictionary.most_common()[:-20:-1]

[('https://t.co/7j2y3rsld9', 1),
 ('@680news', 1),
 ('@torontopearson', 1),
 ('flew', 1),
 ('renewing', 1),
 ('playersã', 1),
 ('subsidise', 1),
 ('non-playing', 1),
 ('premiership', 1),
 ('https://t.co/iaek4fwsgz', 1),
 ('@srinivasiyc', 1),
 ('https://t.co/zjcl195vqs', 1),
 ('#southkerry', 1),
 ('#skelligcoast2kms', 1),
 ('@skelligsix18', 1),
 ('skellig', 1),
 ('milion', 1),
 ('now.when', 1),
 ('https://t.co/5cbliqzx7l', 1)]

**Ответ:** Из плюсов теперь есть только слова без пунктуации и смайликов. Из-за чего уникальных токенов стало меньше. Правда ссылки, хештеги и теги остались.
Из минусов теперь нет пунктуации и смайликов, которые могут быть очень важны для определения настроения :)

Выведите топ-10 популярных хештегов с количеством встреч. Что можно сказать о них?

In [123]:
words = [item.lower() for ls in map(lambda x: tw.tokenize(x), X_train.OriginalTweet) for item in ls]
stopwords_set = set(stopwords.words("english") + list(punctuation))
dictionary_hashtags = Counter(word for word in words if
                              ((word not in stopwords_set) and (
                                          (len(word) > 1) or (len(word) == 1 and (ord(word) < 128))) and word[
                                   0] == "#"))

In [127]:
dictionary_hashtags.most_common(20)

[('#coronavirus', 8808),
 ('#covid19', 2589),
 ('#covid_19', 1734),
 ('#covid2019', 946),
 ('#toiletpaper', 744),
 ('#covid', 641),
 ('#socialdistancing', 465),
 ('#coronacrisis', 448),
 ('#pandemic', 257),
 ('#coronaviruspandemic', 249),
 ('#stayhome', 235),
 ('#coronavirusoutbreak', 223),
 ('#covid-19', 218),
 ('#corona', 209),
 ('#lockdown', 208),
 ('#supermarket', 206),
 ('#stayathome', 197),
 ('#panicbuying', 197),
 ('#stayhomesavelives', 194),
 ('#stophoarding', 190)]

**Ответ:** Ладно кажется хештеги всё-таки несут какой-то смысл. По ним можно определить настрояния. Так, например хештеги #lockdown, #panicbuying или #supermarket скорее будут нести в себе негативный посыл. Но также много и шума, например #covid19, так как все твиты и так связаны с коронавирусом.

То же самое проделайте для ссылок на сайт https://t.co Сравнима ли популярность ссылок с популярностью хештегов? Будет ли информация о ссылке на конкретную страницу полезна?

In [131]:
words = [item.lower() for ls in map(lambda x: tw.tokenize(x), X_train.OriginalTweet) for item in ls]
stopwords_set = set(stopwords.words("english") + list(punctuation))
dictionary_links = Counter(word for word in words if
                           ((word not in stopwords_set) and ((len(word) > 1) or (
                                       len(word) == 1 and (ord(word) < 128))) and "https://t.co" in word))

In [133]:
dictionary_links.most_common(15)

[('https://t.co/oxa7swtond', 5),
 ('https://t.co/gp3eusapl8', 4),
 ('https://t.co/deftrui1pfã\x82â', 3),
 ('https://t.co/wrlhyzizaa', 3),
 ('https://t.co/kuwipf1kqw', 3),
 ('https://t.co/zjnrx6dkkn', 3),
 ('https://t.co/3gbbdpdjat', 3),
 ('https://t.co/e2znxajpre', 3),
 ('https://t.co/catkegayoy', 3),
 ('https://t.co/g63rp042ho', 3),
 ('https://t.co/aziqcdgrnn', 3),
 ('https://t.co/bylqxrjmnt', 3),
 ('https://t.co/wuieefsnoj', 3),
 ('https://t.co/oi39zsanq8', 3),
 ('https://t.co/rafj2l2ceq', 2)]

**Ответ:** Нет, не будет полезна, так как в самих ссылках не хранится никакой информации. К тому же они редко повторяются и только переобучат модель (максимум встречаются 3 раза). Хештеги будут куда полезнее, так как они и содержат информацию, и часто встречаются (многие хештеги встречаются сотни и тысячи раз)

Используем опыт предыдущих экспериментов и напишем собственный токенайзер, улучшив TweetTokenizer. Функция tokenize должна:



*   Привести текст в нижний регистр
*   Применить TweetTokenizer для изначального выделения токенов
*   Удалить стоп-слова, пунктуацию, токены из одного символа, с позицией в таблице Unicode 128 и более и ссылки на t.co



In [164]:
def custom_tokenizer(text): # по сути у меня уже всё было сделано в предыдущих заданиях
    words = map(lambda x: x.lower(), tw.tokenize(text)) # список слов в нижнем регистре
    stopwords_set = set(stopwords.words("english") + list(punctuation)) # стоп-слова и символы пунктуации
    tokens = [word for word in words if
                         ((word not in stopwords_set) and ((len(word) > 1) or (len(word) == 1 and (ord(word) < 128))) and "https://t.co" not in word)] # Список с фильтрацией токенов. Условие, конечно, можно вынести в функцию, но вроде и так нормально

    return tokens


In [165]:
custom_tokenizer('This is sample text!!!! @Sample_text I, \x92\x92 https://t.co/sample  #sampletext')

['sample', 'text', '@sample_text', '#sampletext']

## Задание 3 Векторизация текстов (2 балла)

Обучите CountVectorizer с использованием custom_tokenizer в качестве токенайзера. Как размер полученного словаря соотносится с размером изначального словаря из начала задания 2?

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer  # -- YOUR CODE HERE --

print(len(cv.vocabulary_))

45308


**Ответ:** # -- YOUR ANSWER HERE --

Посмотрим на какой-нибудь конкретный твитт:

In [ ]:
ind = 9023
train.iloc[ind]['OriginalTweet'], train.iloc[ind]['Sentiment']

Автор твитта не доволен ситуацией с едой во Франции и текст имеет резко негативную окраску.

Примените обученный CountVectorizer для векторизации данного текста, и попытайтесь определить самый важный токен и самый неважный токен (токен, компонента которого в векторе максимальна/минимальна, без учета 0). Хорошо ли они определились, почему?

In [ ]:
# -- YOUR CODE HERE --

**Ответ:** # -- YOUR ANSWER HERE --

Теперь примените TfidfVectorizer и  определите самый важный/неважный токены. Хорошо ли определились, почему?

In [ ]:
# -- YOUR CODE HERE --

**Ответ:** # -- YOUR ANSWER HERE --

Найдите какой-нибудь положительно окрашенный твитт, где TfidfVectorizer хорошо (полезно для определения окраски) выделяет важный токен, поясните пример.

*Подсказка:* явно положительные твитты можно искать при помощи положительных слов (good, great, amazing и т. д.)

In [ ]:
train[train['OriginalTweet'].apply(lambda x: 'your_good_word_here' in x) & (train['Sentiment'] == 1)]

,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment


In [ ]:
# -- YOUR CODE HERE --

**Ответ:** # -- YOUR ANSWER HERE --

## Задание 4 Обучение первых моделей (1 балл)

Примените оба векторайзера для получения матриц с признаками текстов.  Выделите целевую переменную.

In [ ]:
# -- YOUR CODE HERE --

Обучите логистическую регрессию на векторах из обоих векторайзеров. Посчитайте долю правильных ответов на обучающих и тестовых данных. Какой векторайзер показал лучший результат? Что можно сказать о моделях?

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# -- YOUR CODE HERE --

**Ответ:** # -- YOUR ANSWER HERE --

## Задание 5 Стемминг (0.5 балла)

Для уменьшения словаря можно использовать стемминг.

Модифицируйте написанный токенайзер, добавив в него стемминг с использованием SnowballStemmer. Обучите Count- и Tfidf- векторайзеры. Как изменился размер словаря?

In [ ]:
def custom_stem_tokenizer(text):
    # -- YOUR CODE HERE --
    return tokens

In [ ]:
custom_stem_tokenizer(
    'This is sample text!!!! @Sample_text I, \x92\x92 https://t.co/sample  #sampletext adding more words to check stemming')

['sampl', 'text', '@sample_text', '#sampletext', 'ad', 'word', 'check', 'stem']

In [ ]:
cv = CountVectorizer  # -- YOUR CODE HERE --

print(len(cv.vocabulary_))

36652


**Ответ** # -- YOUR ANSWER HERE --

Обучите логистическую регрессию с использованием обоих векторайзеров. Изменилось ли качество? Есть ли смысл применять стемминг?

In [ ]:
# -- YOUR CODE HERE --

**Ответ:** # -- YOUR ANSWER HERE --

## Задание  6 Работа с частотами (1.5 балла)

Еще один способ уменьшить количество признаков - это использовать параметры min_df и max_df при построении векторайзера  эти параметры помогают ограничить требуемую частоту встречаемости токена в документах.

По умолчанию берутся все токены, которые встретились хотя бы один раз.



Подберите max_df такой, что размер словаря будет 36651 (на 1 меньше, чем было). Почему параметр получился такой большой/маленький?

In [ ]:
cv_df = CountVectorizer(tokenizer=custom_stem_tokenizer,
                        max_df=  # -- YOUR CODE HERE --
                        ).fit(
    # -- YOUR CODE HERE --
)
print(len(cv_df.vocabulary_))

36651


In [ ]:
# -- YOUR CODE HERE --

**Ответ:** # -- YOUR ANSWER HERE --

Подберите min_df (используйте дефолтное значение max_df) в CountVectorizer таким образом, чтобы размер словаря был 3700 токенов (при использовании токенайзера со стеммингом), а качество осталось таким же, как и было. Что можно сказать о результатах?

In [ ]:
cv_df = CountVectorizer(tokenizer=custom_stem_tokenizer,
                        min_df=  # -- YOUR CODE HERE --
                        ).fit(
    # -- YOUR CODE HERE --
)
print(len(cv_df.vocabulary_))

3700


In [ ]:
# -- YOUR CODE HERE --

**Ответ:** # -- YOUR ANSWER HERE --

В предыдущих заданиях признаки не скалировались. Отскалируйте данные (при словаре размера 3.7 тысяч, векторизованные CountVectorizer), обучите логистическую регрессию, посмотрите качество и выведите `berplot` содержащий по 10 токенов, с наибольшим по модулю положительными/отрицательными весами. Что можно сказать об этих токенах?

In [ ]:
from sklearn.preprocessing import StandardScaler
# -- YOUR CODE HERE --

**Ответ:** # -- YOUR ANSWER HERE --

## Задание 7 Другие признаки (1.5 балла)

Мы были сконцентрированы на работе с текстами твиттов и не использовали другие признаки - имена пользователя, дату и местоположение

Изучите признаки UserName и ScreenName. полезны ли они? Если полезны, то закодируйте их, добавьте к матрице с отскалированными признаками, обучите логистическую регрессию, замерьте качество.

In [ ]:
# -- YOUR CODE HERE --

**Ответ:** # -- YOUR ANSWER HERE --

Изучите признак TweetAt в обучающей выборке: преобразуйте его к типу datetime и нарисуйте его гистограмму с разделением по цвету на оспнове целевой переменной. Полезен ли он? Если полезен, то закодируйте его, добавьте к матрице с отскалированными признаками, обучите логистическую регрессию, замерьте качество.

In [ ]:
# -- YOUR CODE HERE --

**Ответ:** # -- YOUR ANSWER HERE --



Поработайте с признаком Location в обучающей выборке. Сколько уникальных значений?

In [ ]:
# -- YOUR CODE HERE --

Постройте гистограмму топ-10 по популярности местоположений (исключая Unknown)

In [ ]:
# -- YOUR CODE HERE --

Видно, что многие местоположения включают в себя более точное название места, чем другие (Например, у некоторых стоит London, UK; а у некоторых просто UK или United Kingdom).

Создайте новый признак WiderLocation, который содержит самое широкое местоположение (например, из London, UK должно получиться UK). Сколько уникальных категорий теперь? Постройте аналогичную гистограмму.

In [ ]:
# -- YOUR CODE HERE --

Закодируйте признак WiderLocation с помощью OHE таким образом, чтобы создались только столбцы для местоположений, которые встречаются более одного раза. Сколько таких значений?


In [ ]:
# -- YOUR CODE HERE --

Добавьте этот признак к матрице отскалированных текстовых признаков, обучите логистическую регрессию, замерьте качество. Как оно изменилось? Оказался ли признак полезным?


*Подсказка:* используйте параметр `categories` в энкодере.

In [ ]:
# -- YOUR CODE HERE --

**Ответ:** # -- YOUR ANSWER HERE --

## Задание 8 Хорошее качество (Бонус 1 балл)

Добейтесь accuracy=0.9 на тестовой выборке (можно сменить токенайзер, векторайзер, модель и т.д.)

In [ ]:
# -- YOUR CODE HERE --